In [1]:
import pandas as pd

##### Reading data

In [3]:
#filename = 'D:\\Backup\\ISB CBA\\Capstone\\summary data\\transaction_summary.csv'

#data=pd.read_csv(filename)
#d={'quantity':'count','sales_quantity':'sum','purchase_quantity':'sum','transfer_quantity':'sum'}
#chunksize = 10 ** 5
#rows=0
#df_north = pd.DataFrame()
#for chunk in pd.read_csv(filename, chunksize=chunksize):
# 
#    n=chunk[chunk['state']=='ST03']
#    df_north=pd.concat([df_north,n])
#    rows+=chunk.shape[0]
    #print (rows)

In [2]:
filename = 'C:/Users/rohit/ISB/Capstone/Area02 data and model/summary_sorted_area_02.csv'
df_north=pd.read_csv(filename)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_north.shape

(1103480, 51)

In [4]:
df_north.columns

Index(['state', 'location_code', 'item_no', 'week', 'closing_date', 'region',
       'brand', 'quantity', 'purchase_quantity', 'transfer_quantity',
       'sales_quantity', 'purchase_cost_amount', 'purchase_mrp',
       'purchase_date', 'stock_prevailing_mrp', 'store_in',
       'product_group_code', 'transfer_cost_amount', 'sales_department',
       'days_to_sell', 'num_of_customers', 'total_price', 'line_discount',
       'crm_line_discount', 'discount', 'tax', 'cost', 'billing',
       'contribution', 'trade_incentive', 'trade_incentive_value',
       'total_contribution', 'store_type', 'store_location', 'city_type',
       'available_quantity', 'week_no', 'year', 'case_size', 'case_size_range',
       'gender', 'movement', 'material', 'dial_color', 'strap_type',
       'strap_color', 'precious_stone', 'glass', 'case_shape', 'watch_type',
       'area_code'],
      dtype='object')

In [5]:
#extracting specific columns

col=['location_code', 'item_no', 'state', 'region',
       'brand', 'stock_prevailing_mrp', 'store_type', 'store_location', 'city_type',
       'available_quantity', 'week_no', 'year', 'case_size_range',
       'gender', 'movement', 'material', 'dial_color', 'strap_type',
       'strap_color', 'precious_stone', 'glass', 'case_shape', 'watch_type','billing','sales_quantity']

df_north_final=df_north.loc[:,col]
df_north_final.fillna(0, inplace=True)

df_north_final['item_no']=df_north_final['item_no'].astype(str)
#df_north_final['case_size']=df_north_final['case_size'].astype(str)
df_north_final['case_shape']=df_north_final['case_shape'].astype(str)

#### Defining a function for doing pareto analysis on features

The function aggregates all levels of a categorical features that account for ~10% by Sales billings into a new level called "others". Features with less than 10 levels are not considered for pareto analysis.

In [6]:
def pareto(cols):
    lst=[]
    for col in cols:
                
        series=df_north_final.fillna(0).groupby([col]).agg({'billing':'sum'}).sort_values('billing',ascending=False)
        mask=series.cumsum()/series.sum()>0.9 
        #nos=mask.value_counts()[1]
        mask=mask.iloc[:,0]
        levels=len(df_north_final[col].unique())
        if levels>10:
            df_north_final[col] = np.where(df_north_final[col].isin(series[mask].index),'Other',df_north_final[col])         
        new_levels=len(df_north_final[col].unique())

        freq=df_north_final[col].value_counts()/df_north_final[col].value_counts().sum()*100
        freq=freq.round(2)

        sale_qty=df_north_final.groupby([col]).agg({'sales_quantity':'sum'}).sort_values('sales_quantity',ascending=False)
        sale_qty=sale_qty/sale_qty.sum()*100
        sale_qty=sale_qty.round(2)
        try:
            Other_Sales_Qty=sale_qty['sales_quantity']['Other']
        except:
            Other_Sales_Qty=0
        
        mrp=df_north_final.groupby([col]).agg({'stock_prevailing_mrp':'sum'}).sort_values('stock_prevailing_mrp',ascending=False)
        mrp=mrp/mrp.sum()*100
        mrp=mrp.round(2)
        try:
            Other_MRP=mrp['stock_prevailing_mrp']['Other']
        except:
            Other_MRP=0
        
        #comparison=mrp.merge(sale_qty, left_index=True, right_index=True)
        lst.append([col.upper(),levels, new_levels,Other_MRP,Other_Sales_Qty])
        #print ("%s-Originally %d levels,combined %d levels into 'Other'.New Levels %d.By MRP,Other is %2.1f and by sale qty others is %2.1f"%(col.upper(),levels, levels-new_levels, new_levels,mrp['stock_prevailing_mrp']['Other'],sale_qty['sales_quantity']['Other']))
    
    cols=['Feature', 'Orig Levels', 'New Levels', 'Other%(MRP)', 'Other%(Sales Qty)']
    df = pd.DataFrame(lst, columns=cols)
    df=df.set_index("Feature")
    
    return df

In [7]:
import numpy as np
#cols=['brand','case_size', 'case_size_range', 'gender', 'material', 'dial_color', 'strap_type', 'strap_color','precious_stone', 'glass', 'watch_type']
cols=['case_size_range', 'gender','movement', 'material', 'dial_color', 'strap_type', 'strap_color','precious_stone', 'glass', 'case_shape', 'watch_type']
pareto(cols)

,Orig Levels,New Levels,Other%(MRP),Other%(Sales Qty)
Feature,,,,
CASE_SIZE_RANGE,13,6,20.98,22.70
GENDER,3,3,0.00,0.00
MOVEMENT,10,10,0.00,0.00
MATERIAL,80,10,14.10,15.51
DIAL_COLOR,63,11,13.27,10.56
STRAP_TYPE,81,11,12.30,14.97
STRAP_COLOR,56,7,15.45,21.41
PRECIOUS_STONE,9,9,0.00,0.00
GLASS,8,8,0.00,0.00


### Modelling

In [8]:
#sorting by week_no and year
df_north_final=df_north_final.sort_values(['week_no', 'year'], ascending=[True, False])

In [9]:
#creatign dummy variables
cols=['brand','state','region', 'store_type', 'store_location', 'city_type',
       'case_size_range', 'gender', 'movement', 'material', 'dial_color',
       'strap_type', 'strap_color', 'precious_stone', 'glass', 'case_shape',
       'watch_type', 'week_no', 'year']
df_north_dummies=pd.get_dummies(data=df_north_final, columns=cols)

In [10]:
df_north_dummies.shape

(1103480, 274)

In [11]:
#!pip install xgboost
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [12]:
df_north_dummies.head()

,location_code,item_no,stock_prevailing_mrp,available_quantity,billing,sales_quantity,brand_0,brand_B003,brand_B007,brand_B008,...,week_no_W47,week_no_W48,week_no_W49,week_no_W50,week_no_W51,week_no_W52,week_no_W53,year_FY1718,year_FY1819,year_FY1920
13053,S03,5127095,24990.0,1,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
13054,S03,5154203,196000.0,1,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
13055,S03,5185559,111000.0,1,0.0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
13056,S03,5188116,252000.0,3,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
13057,S03,5188553,8295.0,1,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [33]:
df_north_dummies['sales_quantity'].unique()

array([  0,   1,   7,   3,   2,  -1,   4,   5,  -2,  -4,   9,  12,   6,
        -3,  10,   8,  20,  11,  14,  -5, 135, 278,  64,  28,  22,  17,
        13,  15,  -7,  30, 100,  55,  23,  83, 227], dtype=int64)

In [13]:
y=df_north_dummies.loc[:, df_north_dummies.columns == 'sales_quantity']
X=df_north_dummies.drop(columns =['sales_quantity','location_code','item_no'])

In [14]:
#checking for duplicate column names
duplicate_columns = df_north_dummies.columns[df_north_dummies.columns.duplicated()]
duplicate_columns

Index([], dtype='object')

In [15]:
#performing train and test split on data

split=0.2
test=int(len(X)*split)
train=len(X)-test
X_train=X.head(train)
y_train=y.head(train)
X_test=X.tail(test)
y_test=y.tail(test)

#### Modelling using linear regression

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import accuracy_score

In [17]:
LRmodel = LinearRegression()  
LRmodel.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [27]:
R2_score=LRmodel.score(X_train, y_train)
R2_score

0.15015266330264176

In [19]:
y_pred = LRmodel.predict(X_test)

In [23]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse

0.5732076056248514

In [34]:
df1 = pd.DataFrame({'Actual': y_test.values.flatten(), 'Predicted': y_pred.flatten()})
df1.head()

,Actual,Predicted
0,0,0.009260
1,0,0.041611
2,0,-0.062587
3,0,0.017291
4,0,0.157778


In [36]:
df1.to_excel('linear_actual_predicted.xlsx')

#### Modelling using logistic regression

In [40]:
df_north_dummies['sales'] = np.where(df_north_dummies['sales_quantity']>0, 1, 0)

In [41]:
df_north_dummies.head()

,location_code,item_no,stock_prevailing_mrp,available_quantity,billing,sales_quantity,brand_0,brand_B003,brand_B007,brand_B008,...,week_no_W48,week_no_W49,week_no_W50,week_no_W51,week_no_W52,week_no_W53,year_FY1718,year_FY1819,year_FY1920,sales
13053,S03,5127095,24990.0,1,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
13054,S03,5154203,196000.0,1,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
13055,S03,5185559,111000.0,1,0.0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
13056,S03,5188116,252000.0,3,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
13057,S03,5188553,8295.0,1,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [42]:
y=df_north_dummies.loc[:, df_north_dummies.columns == 'sales']
X=df_north_dummies.drop(columns =['sales_quantity','location_code','item_no'])

In [43]:
#performing train and test split on data

split=0.2
test=int(len(X)*split)
train=len(X)-test
X_train=X.head(train)
y_train=y.head(train)
X_test=X.tail(test)
y_test=y.tail(test)

In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import accuracy_score

In [45]:
logistic = LogisticRegression()
logistic.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [46]:
y_pred = logistic.predict(X_test)
df2 = pd.DataFrame({'Actual': y_test.values.flatten(), 'Predicted': y_pred.flatten()})

In [47]:
predictions = [round(value) for value in y_pred]

In [48]:
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Logistic Regression Accuracy: %.2f%%" % (accuracy * 100.0))

Logistic Regression Accuracy: 98.99%


In [49]:
df2.to_excel('logistic_actual_predicted.xlsx')

#### Modelling using xgboost

In [37]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,max_depth = 5, alpha = 10, n_estimators = 10)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.577194


In [38]:
from sklearn.metrics import r2_score
r2_score(y_test,preds)

-0.04497181985992138

##### Tuning hyperparameters

In [ ]:
params = {'colsample_bytree': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
          'subsample': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
          'colsample_bylevel':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
          'min_child_weight ':[1,3,5,7] ,
          'reg_lambda':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
          'n_estimator':[3,5,7,10],
          'learning_rate': [0.05,0.1,0.15,0.2,0.25,0.3],
          'max_depth': [3,4,5,6,7,8,10,12,15], 
          'reg_alpha': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
         }

In [ ]:
params = {'colsample_bytree': [0.1,0.2],
          #'subsample': [0.1,0.2,0.3,0.4],
          #'colsample_bylevel':[0.1,0.2,0.3,0.4],
          #'min_child_weight ':[1,3,5,7] ,
          'reg_lambda':[0.1,0.2,0.3]
          #'n_estimator':[3,5,],
          #'learning_rate': [0.05,0.1,0.15,0.2,0.25,0.3],
          #'max_depth': [3,4,5,6,7,8,10,12,15], 
          #'reg_alpha': [0.1,0.2,0.3]
         }

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror')
scoring='r2'
#scoring='neg_mean_squared_error'
random_search=RandomizedSearchCV(xg_reg,param_distributions=params,scoring=scoring,n_iter=20,cv=5,verbose=3)

In [ ]:
random_search.fit(X_train,y_train)

In [ ]:
random_search.best_estimator_

In [ ]:
random_search.best_params_

In [50]:
best_gb=xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.4,
             colsample_bynode=1, colsample_bytree=0.9, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.2, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=np.nan,
             monotone_constraints='()', n_estimator=5, n_estimators=100,
             n_jobs=0, num_parallel_tree=1, objective='reg:squarederror',
             random_state=0, reg_alpha=0.4, reg_lambda=0.1, scale_pos_weight=1,
             subsample=0.3, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [44]:
from sklearn.model_selection import cross_val_score as cvs
score=cvs(best_gb,X_train,y_train,cv=5,scoring='r2')

[17:10:03] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { n_estimator } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:13:06] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { n_estimator } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:15:59] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { n_estimator } might not be used.

  This may not be accurate due to some parameters are only used in langu

C:\Users\Raja Sekhar\Anaconda33\lib\site-packages\sklearn\model_selection\_validation.py:530: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


MemoryError: 

In [45]:
score

array([0.49383491, 0.31226366, 0.29071351, 0.3636798 , 0.29428447])

In [46]:
score.mean()

0.3509552712445454

In [52]:
best_gb.fit(X_train,y_train)
preds = best_gb.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

[17:42:35] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { n_estimator } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


RMSE: 0.623945


In [53]:
from sklearn.metrics import r2_score
r2_score(y_test,preds)

0.0802965281857646